# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [5]:
driver = webdriver.Chrome()

In [6]:
driver.get('https://www.tdlr.texas.gov/tools_search/')

In [7]:
text_input = driver.find_element_by_id('mcrdata')

In [8]:
text_input.send_keys('006179570C')

In [9]:
button = driver.find_element_by_xpath('//*[@id="submit3"]/b')
button.click()

In [10]:
name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text

In [11]:
owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text

In [12]:
phone_no = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text

In [13]:
lisence_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font').text

In [14]:
address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text

In [15]:
address

'Carrier Type:  Tow Truck Company\nNumber of Active Tow Trucks:   0\n\nAddress Information\nMailing:\n13619 BRETT JACKSON RD\nFORT WORTH, TX. 76179\n\nPhysical:\n13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179'

In [16]:
address[-46:]

'\n13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179'

In [17]:
address_info = address.split(" ")[-12:-7]
address_info

['BRETT', 'JACKSON', 'RD\nFORT', 'WORTH,', 'TX.']

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [18]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C')
driver.get('https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C')

name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
phone_no = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
lisence_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font').text
address_info = address.split(" ")[-12:-7]

print(name)
print(owner)
print(phone_no)
print(lisence_status)
print(address_info)



Name:   B.D. SMITH TOWING
Owner/Officer:   BRANDT SMITH / OWNER
Phone:   8173330706
Expired
['BRETT', 'JACKSON', 'RD\nFORT', 'WORTH,', 'TX.']


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [19]:
df = pd.read_csv('trucks-subset.csv')

## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [28]:
df.head()

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


In [39]:
def get_url_info(row):
    
    url = "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + row['TDLR Number']
    
    return url 

    
    
    


In [40]:
pd.set_option('display.max_colwidth', -1)

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [43]:
df['URL'] = df.apply(get_url_info, axis=1)


In [44]:
df.head()

,TDLR Number,URL
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [55]:
def get_info_tow_truck(row):

    driver.get(row['URL'])

    name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    phone_no = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    lisence_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font').text
    address_info = address.split(" ")[-12:-7]

    return pd.Series({
        'name': name,
        'owner': owner,
        'phone': phone_no,
        'status':lisence_status,
        'address': address_info
    }) 



In [56]:
new_df = df.head().apply(get_info_tow_truck, axis=1).join(df)
new_df

,address,name,owner,phone,status,TDLR Number,URL
0,"[BRETT, JACKSON, RD\nFORT, WORTH,, TX.]",Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Active,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,"[BRETT, JACKSON, RD\nFORT, WORTH,, TX.]",Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Expired,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,"[BRETT, JACKSON, RD\nFORT, WORTH,, TX.]",Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Active,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [57]:
new_df.to_csv("tow-trucks-extended.csv", index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [58]:
new_df.head()

,address,name,owner,phone,status,TDLR Number,URL
0,"[BRETT, JACKSON, RD\nFORT, WORTH,, TX.]",Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Active,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,"[BRETT, JACKSON, RD\nFORT, WORTH,, TX.]",Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Expired,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,"[BRETT, JACKSON, RD\nFORT, WORTH,, TX.]",Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Active,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [59]:
df = pd.read_csv('tow-trucks.csv')

In [60]:
df.head()

,TDLR Number
0,006507931C
1,006179570C
2,006502097C
3,006494912C
4,0649468VSF


In [61]:
def get_url_info(row):
    
    url = "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + row['TDLR Number']
    
    return url 


In [62]:
df['URL'] = df.apply(get_url_info, axis=1)

In [63]:
def get_info_tow_truck(row):

    driver.get(row['URL'])

    name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    phone_no = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    lisence_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font').text
    address_info = address.split(" ")[-12:-7]

    return pd.Series({
        'name': name,
        'owner': owner,
        'phone': phone_no,
        'status':lisence_status,
        'address': address_info
    }) 


In [64]:
new_df_2 = df.head().apply(get_info_tow_truck, axis=1).join(df)
new_df_2

,address,name,owner,phone,status,TDLR Number,URL
0,"[BRETT, JACKSON, RD\nFORT, WORTH,, TX.]",Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Active,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,"[BRETT, JACKSON, RD\nFORT, WORTH,, TX.]",Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Expired,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,"[BRETT, JACKSON, RD\nFORT, WORTH,, TX.]",Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Active,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,"[BRETT, JACKSON, RD\nFORT, WORTH,, TX.]",Name: HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 940-552-0687,Expired,006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,"[BRETT, JACKSON, RD\nFORT, WORTH,, TX.]",Name: HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 9405520687,Expired,0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF


In [65]:
new_df_2.to_csv("tow-trucks.csv", index=False)